## Exploring a THREDDS catalog with Unidata's Siphon

In [1]:
from siphon.catalog import TDSCatalog

catalog = TDSCatalog('http://thredds.cencoos.org/thredds/catalog.xml')

### Siphon return some useful attributes.

In [2]:
info = """
Catalog information
-------------------

Base THREDDS URL: {}
Catalog name: {}
Catalog URL: {}
Metadata: {}
""".format(catalog.base_tds_url,
           catalog.catalog_name,
           catalog.catalog_url,
           catalog.metadata)

print(info)


Catalog information
-------------------

Base THREDDS URL: http://thredds.cencoos.org
Catalog name: CeNCOOS
Catalog URL: http://thredds.cencoos.org/thredds/catalog.xml
Metadata: {}



### Sadly this dataset had no metadata :-(
### What kind of services are avialable?

In [3]:
for service in catalog.services:
    print(service.name)

all
allandsos
wms


### The catalog refs...

In [4]:
print('\n'.join(catalog.catalog_refs.keys()))

Global
Dynamic
Static
HF RADAR, US West Coast
HF RADAR, US West Coast (GNOME Format)


### ... and datasets.

In [5]:
print('\n'.join(catalog.datasets.keys()))

California Coastal Regional Ocean Modeling System (ROMS) Nowcast
California Coastal Regional Ocean Modeling System (ROMS) Forecast
Monterey Bay (MB) Regional Ocean Modeling System (ROMS) Nowcast
Monterey Bay (MB) Regional Ocean Modeling System (ROMS) Forecast
Southern California Bight (SCB) Regional Ocean Modeling System (ROMS) Nowcast
UCSC California Current System Model
HAB Cellular Domoic Acid Forecast
HAB Cellular Domoic Acid Nowcast
HAB Particulate Domoic Acid Forecast
HAB Particulate Domoic Acid Nowcast
HAB Pseudo Nitzschia Forecast
HAB Pseudo Nitzschia Nowcast
Coupled Ocean/Atmosphere Mesoscale Prediction System (COAMPS) Relative Humidity
Coupled Ocean/Atmosphere Mesoscale Prediction System (COAMPS) Total Precipitation
Coupled Ocean/Atmosphere Mesoscale Prediction System (COAMPS) Visibility
Coupled Ocean/Atmosphere Mesoscale Prediction System (COAMPS) Wind 10 m
Coupled Ocean/Atmosphere Mesoscale Prediction System (COAMPS) Air Temp 2 m
Coupled Ocean/Atmosphere Mesoscale Predictio

### What is a catalog ref?

In [6]:
ref = catalog.catalog_refs['Global']

[value for value in dir(ref) if not value.startswith('__')]

['follow', 'href', 'name', 'title']

In [7]:
info = """
Href: {}
Name: {}
Title: {}
""".format(
    ref.href,
    ref.name,
    ref.title)

print(info)


Href: http://thredds.cencoos.org/thredds/global.xml
Name: 
Title: Global



### Those are attributes holding the metadata of that ref. Let's take a look at the `follow` is a method.

In [8]:
cat = ref.follow()

print(type(cat))

<class 'siphon.catalog.TDSCatalog'>


### We followed that `ref` to a new catalog! Here are the data from the *Global* catalog object.

In [9]:
print('\n'.join(cat.datasets.keys()))

NCEP Reanalysis Daily Averages Surface Flux
Global 1-km Sea Surface Temperature (G1SST)
NCEP Global Forecast System Model (GFS)
Aquarius V 3.0 Scatterometer Daily Aggregate
Aquarius V 3.0 Scatterometer Seven-Day Aggregate
Aquarius V 3.0 Scatterometer Monthly Aggregate
Aquarius V 3.0 Radiometer Daily Aggregate
Aquarius V 3.0 Radiometer Seven-Day Aggregate
Aquarius V 3.0 Radiometer Monthly Aggregate
Aquarius V 4.0 Scatterometer Daily Aggregate
Aquarius V 4.0 Scatterometer Seven-Day Aggregate
Aquarius V 4.0 Scatterometer Monthly Aggregate
Aquarius V 4.0 Radiometer Daily Aggregate
Aquarius V 4.0 Radiometer Seven-Day Aggregate
Aquarius V 4.0 Radiometer Monthly Aggregate


### Let's extract the Global SST from both the main catalog and the "subcatalog."

In [10]:
dataset = 'Global 1-km Sea Surface Temperature (G1SST)'

ds0 = catalog.datasets[dataset]
ds1 = cat.datasets[dataset]

ds0 is ds1, ds0 == ds1

(False, False)

### Not sure if they are really different datasets or if siphon just spawn a new instance that comparison is bogus. Let's check the metadata.

In [11]:
ds0.url_path, ds1.url_path

('G1_SST_US_WEST_COAST.nc', 'G1_SST_GLOBAL.nc')

### Aha! They are different datasets! Siphon has a `ncss` (NetCDF subset service) too. Here is the docstring:

> This module contains code to support making data requests to
the NetCDF subset service (NCSS) on a THREDDS Data Server (TDS). This includes
forming proper queries as well as parsing the returned data.

### But it seems that the catalog must offer the `NetcdfSubset` in the access_urls. Let's see if we have that in this catalog.

In [12]:
for name, ds in catalog.datasets.items():
    if ds.access_urls:
        print(name)

### All empty... Maybe that is just a metadata issue because we can see `NetcdfSubset` in the page.

In [13]:
from IPython.display import IFrame

url = 'http://thredds.cencoos.org/thredds/catalog.html?dataset=G1_SST_US_WEST_COAST'

IFrame(url, width=800, height=550)

### So let's try something else for now. We can get access the WMS service and overlay the data in a slippy map.

In [14]:
services = [service for service in catalog.services if service.name == 'wms']

services

### Found only one. So we can squeeze that out.

In [15]:
service = services[0]

In [16]:
url = service.base
url

'http://pdx.axiomalaska.com/ncWMS/wms'

We do not know what is avilable there but we can explore the service with `owslib`.

In [17]:
from owslib.wms import WebMapService

service = WebMapService(url)

print('\n'.join(service.contents.keys()))

NDFD_2016_Total_Snowfall/Total_snowfall
NDFD_2016_Minimum_Temperature/Minimum_temperature
NDFD_2016_Significant_height_of_wind_waves/Significant_height_of_wind_waves
NDFD_2016_Total_Precipitation/Total_precipitation
NDFD_2016_Wind_Speed_Gust/Wind_speed_gust
NDFD_2016_Maximum_Temperature/Maximum_temperature
NDFD_2016_Aggregate/Wind_direction_from_which_blowing
NDFD_2016_Aggregate/Wind_speed
NDFD_2016_Aggregate/Temperature_tendency_by_all_radiation
NDFD_2016_Aggregate/Relative_humidity
NDFD_2016_Aggregate/Total_cloud_cover
NDFD_2016_Aggregate/Dew_point_temperature
NDFD_2016_Aggregate/Temperature
NDFD_2016_Aggregate/eastward_wind
NDFD_2016_Aggregate/northward_wind
NDFD_2016_Aggregate/wind
COAST_WATCH_JPLMURSST41CLIM/mean_sst
COAST_WATCH_JPLMURSST41ANOM1DAY/sstAnom
COAST_WATCH_JPLMURSST41ANOMMDAY/sstAnom
COAST_WATCH_JPLMURSST41/analysed_sst
WEST_COAST_WAVE_1/U-component_of_wind
WEST_COAST_WAVE_1/Significant_height_of_swell_waves
WEST_COAST_WAVE_1/Significant_height_of_wind_waves
WEST_COAST

Getting the first one from the service list for our example.

In [18]:
layer = list(service.contents.keys())[0]

wms = service.contents[layer]

lon = (wms.boundingBox[0] + wms.boundingBox[2]) / 2.
lat = (wms.boundingBox[1] + wms.boundingBox[3]) / 2.

In [19]:
import folium

m = folium.Map(location=[lat, lon], zoom_start=3)

folium.WmsTileLayer(name='{} at {}'.format(wms.title,
                                           wms.defaulttimeposition),
                    url=url,
                    layers=layer,
                    format='image/png').add_to(m)

folium.LayerControl().add_to(m)

m